# ML MODEL CREATION

Importing the necessary libraries

In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split

Importing the Data

In [12]:
df = pd.read_excel("Data/ML_Data.xlsx")

Splitting training and testing Data

In [13]:
# Splitting the dataset into the features and the target variable
X = df.drop('depression_status', axis=1)  # Features
y = df['depression_status']  # Target variable

# Splitting the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Training the models

In [14]:
X_train.shape

(318, 39)